# **RBC Classification using CNN**

**Importing the required libraries and modules**

In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [8]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

**Setting the required directories etc**

In [9]:
img_width = 150
img_height = 150

train_data_dir = '../input/rbc-corrected/rbc_augmentedNKT/training'
validation_data_dir = '../input/rbc-corrected/rbc_augmentedNKT/validation'
train_samples = 3237
validation_samples = 238
epochs = 90
batch_size = 20

#Check for TensorFlow or Thieno
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

**Creating the model**

In [10]:
from keras.layers import BatchNormalization

In [11]:
model = Sequential()
# Conv2D : Two dimenstional convulational model.
# 32 : Input for next layer
# (3,3) convulonational windows size
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [12]:
import keras
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.Adam(lr=.0005), 
              metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [16]:
# augmentation configuration used for testing:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [18]:
print(train_generator.class_indices)

In [19]:
imgs, labels = next(train_generator)

In [ ]:
#input_shape = (150,150,3)

In [20]:
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

In [21]:
import matplotlib.pyplot as plt
%matplotlib inline
image_batch,label_batch = train_generator.next()

print(len(image_batch))
for i in range(0,len(image_batch)):
    image = image_batch[i]
    print(label_batch[i])
    imshow(image)

In [22]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

**Training the model**

In [23]:
#In fit_generator(), you don't pass the x and y directly, instead they come from a generator.
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

In [24]:
y = model.save_weights('RBC_one.h5') 

In [25]:
x = history.history['accuracy']
best_accuracy = max(x)
best_accuracy

**Final accuracy obtained**

In [26]:
z = history.history['accuracy'][-1]

In [27]:
print('accuracy obtained =',z*100)

In [28]:
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
# list all data in history
print(history.history.keys())
# summarizing the history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarizing the history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Prediction**

In [30]:
predict_dir_path='../input/rbc-corrected/rbc_augmentedNKT/test/'
onlyfiles = [f for f in listdir(predict_dir_path) if isfile(join(predict_dir_path, f))]
print(onlyfiles)

In [35]:
# predicting images
from keras.preprocessing import image
circular_counter = 0 
elongated_counter  = 0
other_counter = 0

for file in onlyfiles:
    img = image.load_img(predict_dir_path +file , target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0]
    
    if classes == 0:
        print(file + ": " + 'circular')
        circular_counter += 1
    
    elif classes == 1:
        print(file + ": " + 'elongated')
        elongated_counter += 1
    else:
        print(file + ": " + 'other')
        other_counter += 1
print("Total circular :", circular_counter)
print("Total elongated :",elongated_counter)
print("Total Other :",other_counter)